In [8]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

# 5.2 連鎖律
- 合成関数の微分の性質
- 高速に勾配を計算する誤差逆伝播法に応用する
- ニューラルネット全体の関数を，各ノードの局所的な関数とその先の様々な関数の合成関数と考えて応用

$$
z = f(t) \\
t = g(x, y)
$$
とすると，次のように合成関数の微分を求めることができる
$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

***

### 単純な乗算レイヤ，加算レイヤ

- 単純な乗算レイヤの例．
- 様々な複雑な演算を含む損失関数を $ L(z) $ とする．
- その内，一つの乗算ノードで $ z = xy $ という演算が行われるとする．

このとき，次のように勾配を求めることができる
$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial x} = \frac{\partial L}{\partial z} y \\　\\
\frac{\partial L}{\partial y} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial y} = \frac{\partial L}{\partial z} x
$$
- $ \frac{\partial L}{\partial z} $ は乗算ノードの出力 $ z $ での $ L $ の偏微分．
- $ x, y $ を勾配計算に用いるため，順伝播の入力を保持しておく必要がある．

***

In [11]:
# class MulLayer:
#     def __init__(self):
#         self.x = None
#         self.y = None
        
#     def forward(self, x, y):
#         self.x = x
#         self.y = y
#         out = x * y
#         return out
    
#     def backward(self, dout):
#         dx = dout * self.y
#         dy = dout * self.x
#         return dx, dy

In [13]:
# class AddLayer:
#     def __init__(self):
#         self.x = None
#         self.y = None
        
#     def forward(self, x, y):
#         self.x = x
#         self.y = y
#         out = x + y
        
#         return out
    
#     def backward(self, dout):
#         dx = dout
#         dy = dout
        
#         return dx, dy

# 5.5 活性化関数レイヤ

### ReLUレイヤ
#### ReLU
$$
y = 
\begin{cases}
    x \quad (x \gt 0) \\
    0 \quad (x \leq 0)
\end{cases}
$$

#### ReLUの偏微分
$$
\frac{\partial y}{\partial x} = 
\begin{cases}
    1 \quad (x \gt 0) \\
    0 \quad (x \leq 0)
\end{cases}
$$

***

In [15]:
# class Relu:
#     def __init__(self):
#         self.mask = None
        
#     def forward(self, x):
#         # 入力が 0 以下のインデックスを保持し，その値は0とする
#         self.mask = (x <= 0)
#         out = x.copy()
#         out[self.mask] = 0
        
#         return out
    
#     def backward(self, dout):
#         # 順伝播の入力が 0 以下であったインデックスの値は 0 ，そのほかはそのまま逆伝播
#         dout[self.mask] = 0
#         dx = dout
        
#         return dx

### Sigmoidレイヤ
#### Sigmoid
$$
y = \frac{1}{1 + \exp(-x)}
$$

#### Sigmoidの偏微分導出
$$
t = 1 + \exp(-x)
$$

とおくと，
$$
y = \frac{1}{1 + \exp(-x)} = \frac{1}{t} = t^{-1}
$$

また，
$$
\begin{align}
\frac{\partial y}{\partial t} &= -t^{-2} = -(1 + \exp(-x))^{-2}\\
\frac{\partial t}{\partial x} &= -\exp(-x)
\end{align}
$$

であるから，
$$
\begin{align}
\frac{\partial y}{\partial x} &= \frac{\partial y}{\partial t} \frac{\partial t}{\partial x} \\ \\
&= \frac{\exp(-x)}{\{1 + \exp(-x)\}^{2}} \\ \\
&= \frac{\exp(-x)}{1 + \exp(-x)} \frac{1}{1 + \exp(-x)} \\ \\
&= \left(\frac{1 + \exp(-x)}{1 + \exp(-x)} - \frac{1}{1 + \exp(-x)} \right) \frac{1}{1 + \exp(-x)} \\ \\
&= (1 - y) y
\end{align}
$$

***

In [16]:
# class Sigmoid:
#     def __init__(self):
#         self.out = None
        
#     def forward(self, x):
#         out = 1 / (1 + np.exp(-x))
#         self.out = out
        
#         return out
    
#     def backward(self, dout):
#         # 逆伝播の入力に，Sigmoidの偏微分をかける
#         dx = dout * (1.0 - self.out) * self.out
        
#         return dx

# 5.6 Affine / Softmaxレイヤ

### Affine変換
$$
\boldsymbol{y} = \boldsymbol{x} \cdot W + \boldsymbol{b}
$$

- $\boldsymbol{y}, \ \boldsymbol{x}, \ \boldsymbol{b}$ はベクトル，$W$ は行列
- 順伝播における，入力 $ \boldsymbol{x} $，重み $ W $，バイアス $ \boldsymbol{b} $ を表す
- 損失関数 $ L(Y) (\in \boldsymbol{R}) $ の変数と考える

***
#### Affineレイヤの逆伝播
$$
\frac{\partial L}{\partial \boldsymbol{x}} = \frac{\partial L}{\partial \boldsymbol{y}} W^T \\ \\
\frac{\partial L}{\partial W} = \boldsymbol{x}^T \frac{\partial L}{\partial \boldsymbol{y}} \\ \\
$$

***

#### 導出
- 第 $(l-1)$ 層から第 $l$ 層への変換を考える
- 以下のように定義

$$
\begin{align}
\boldsymbol{y} &= \left( y_1, y_2, \cdots, y_j, \cdots, y_m \right)
\\ \\
\boldsymbol{x} &= \left( x_1, x_2, \cdots, x_k, \cdots, x_n \right)
\\ \\
W &= \begin{pmatrix}
w_{11} & w_{21} & \cdots & w_{m1} \\
w_{12} & w_{22} & \cdots & w_{m2} \\
\vdots & \vdots & w_{jk} & \vdots \\
w_{1n} & w_{2n} & \cdots & w_{mn}
\end{pmatrix}
\\ \\
\boldsymbol{b} &= \left( b_1, b_2, \cdots, b_j, \cdots, b_m \right)
\end{align}
$$
- $n$: $(l-1)$ 層のノード数
- $m$: $l$ 層のノード数
- $\boldsymbol{y}$: Affine変換後の $l$ 層への入力．
- $\boldsymbol{x}$: $(l-1)$ 層からの出力．
- $W$: 重み．$w_{jk}$ は $(l-1)$ 層 $k$ 番目のノードから $l$ 層 $j$ 番目のノードへの重み．
- $\boldsymbol{b}$: バイアス．

***

Affine変換を成分表示

$$\boldsymbol{y} = \left( x_1, x_2, \cdots, x_k, \cdots, x_n \right)
\begin{pmatrix}
w_{11} & w_{21} & \cdots & w_{m1} \\
w_{12} & w_{22} & \cdots & w_{m2} \\
\vdots & \vdots & w_{jk} & \vdots \\
w_{1n} & w_{2n} & \cdots & w_{mn}
\end{pmatrix}
+ \left( b_1, b_2, \cdots, b_j, \cdots, b_m \right)
$$

$\boldsymbol{y}$ の成分 $y_j$

$$
\begin{align}
y_j 
&= x_1 w_{j1} + x_2 w_{j2} + \cdots  + x_n w_{jn} + b_j \\ \\
&= \Sigma_{k=1}^{n} x_k w_{jk} + b_j
\end{align}
$$
- $y_j$ は $x_k, \ w_{jk}, \ b_j, (k = 1, 2, \cdots, n)$ の関数
- $W, \ \boldsymbol{b}$ の $j$ 列目以外の成分は関係ない


$\boldsymbol{y}$ の成分 $y_j$ の偏微分

$$
\begin{align}
\frac{\partial y_j}{\partial x_k} 
&= w_{jk}
\\ \\
\frac{\partial y_j}{\partial w_{jk}} 
&= x_k
\\ \\
\frac{\partial y_j}{\partial b_j} 
&= 1
\end{align}
$$

***

損失関数の $L$ の $\boldsymbol{x}$ による微分

$$
\begin{align}
\frac{\partial L}{\partial x_k}
&= \frac{\partial L}{\partial y_1} \frac{\partial y_1}{\partial x_k} + \frac{\partial L}{\partial y_2} \frac{\partial y_2}{\partial x_k} + \cdots + \frac{\partial L}{\partial y_m} \frac{\partial y_m}{\partial x_k}
\\ \\
&= \left( \frac{\partial L}{\partial y_1}, \frac{\partial L}{\partial y_2}, \cdots, \frac{\partial L}{\partial y_m} \right)
\left(
\begin{array}{c}
\frac{\partial y_1}{\partial x_k} \\
\frac{\partial y_2}{\partial x_k} \\
\vdots \\
\frac{\partial y_m}{\partial x_k}
\end{array}
\right)
\\ \\
&= \frac{\partial L}{\partial \boldsymbol{y}}
\left(
\begin{array}{c}
w_{1k}\\
w_{2k} \\
\vdots \\
w_{mk}
\end{array}
\right)
\end{align}
$$

$k = 1, 2, \cdots, n$ をまとめてベクトルにする

$$
\begin{align}
\frac{\partial L}{\partial \boldsymbol{x}}
&= \left(
\frac{\partial L}{\partial x_1}, \frac{\partial L}{\partial x_2}, \cdots, \frac{\partial L}{\partial x_n}
\right)
\\ \\
&= \frac{\partial L}{\partial \boldsymbol{y}}
\left(
\begin{array}{c}
w_{11} & w_{12} & \cdots & w_{1n} \\
w_{21} & w_{22} & \cdots & w_{2n} \\
\vdots & \vdots & & \vdots \\
w_{m1} & w_{m2} & \cdots & w_{mn}
\end{array}
\right)
\\ \\
&= \frac{\partial L}{\partial \boldsymbol{y}} W^T
\end{align}
$$

***

損失関数の $L$ の $W$ による微分

$$
\begin{align}
\frac{\partial L}{\partial w_{jk}}
&= \frac{\partial L}{\partial y_j} \frac{\partial y_j}{\partial w_{jk}}
\\ \\
&= \frac{\partial L}{\partial y_j} x_k
\end{align}
$$

$ j = 1, 2, \cdots, m, k = 1, 2, \cdots, n $ をまとめて行列にする

$$
\begin{align}
\frac{\partial L}{\partial W}
&= \begin{pmatrix}
\frac{\partial L}{\partial w_{11}} & \frac{\partial L}{\partial w_{21}} & \cdots &\frac{\partial L}{\partial w_{m1}} \\
\frac{\partial L}{\partial w_{12}} & \frac{\partial L}{\partial w_{22}} & \cdots &\frac{\partial L}{\partial w_{m2}} \\
\vdots & \vdots & & \vdots \\
\frac{\partial L}{\partial w_{1n}} & \frac{\partial L}{\partial w_{2n}} & \cdots &\frac{\partial L}{\partial w_{mn}}
\end{pmatrix}
\\ \\
&= \begin{pmatrix}
\frac{\partial L}{\partial y_1} x_1 & \frac{\partial L}{\partial y_2} x_1 & \cdots &\frac{\partial L}{\partial y_m} x_1 \\
\frac{\partial L}{\partial y_1} x_2 & \frac{\partial L}{\partial y_2} x_2 & \cdots &\frac{\partial L}{\partial y_m} x_2 \\
\vdots & \vdots & & \vdots \\
\frac{\partial L}{\partial y_1} x_n & \frac{\partial L}{\partial y_2} x_n & \cdots &\frac{\partial L}{\partial y_m} x_n \\
\end{pmatrix}
\\ \\
&= \left( \begin{array}{c}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{array} \right)
\left( \frac{\partial L}{\partial y_1}, \frac{\partial L}{\partial y_2}, \cdots, \frac{\partial L}{\partial y_m} \right)
\\ \\
&= \boldsymbol{x}^T \frac{\partial L}{\partial \boldsymbol{y}}
\end{align}
$$

***

補足

$\boldsymbol{v}_j = \left( w_{j1}, w_{j2}, \cdots, w_{jn} \right)^T$ とおくと
$W = \left( \boldsymbol{v}_1, \boldsymbol{v}_2, \cdots, \boldsymbol{v}_m \right)$

$$
\begin{align}
\frac{\partial L}{\partial W}
&= \left( \frac{\partial L}{\partial \boldsymbol{v}_1}, \frac{\partial L}{\partial \boldsymbol{v}_2}, \cdots, \frac{\partial L}{\partial \boldsymbol{v}_m} \right)
\\ \\
&= \left( \frac{\partial L}{\partial \boldsymbol{y}_1} \frac{\partial \boldsymbol{y}_1}{\partial \boldsymbol{v}_1}, \frac{\partial L}{\partial \boldsymbol{y}_2} \frac{\partial \boldsymbol{y}_2}{\partial \boldsymbol{v}_2}, \cdots, \frac{\partial L}{\partial \boldsymbol{y}_m} \frac{\partial \boldsymbol{y}_m}{\partial \boldsymbol{v}_m} \right)
\end{align}
$$

***

#### バッチ対応Affineレイヤの実装
- 重み $W$ と，バイアス $\boldsymbol{b}$ を渡してインスタンスを生成
- 入力 $\boldsymbol{x}$ を渡して順伝播
- $\frac{\partial L}{\partial \boldsymbol{y}}$ を渡して逆伝播
- $\frac{\partial L}{\partial \boldsymbol{b}}$ はaxis=0で総和を求める

***

In [17]:
# class Affine:
#     def __init__(self, W, b):
#         self.W = W
#         self.b = b
#         self.x = None
#         self.dW = None
#         self.db = None
        
#     def forward(self, x):
#         self.x = x
#         out = np.dot(x, self.W) + self.b
        
#         return out
    
#     def backward(self, dout):
#         dx = np.dot(dout, self.W.T)
#         self.dW = np.dot(self.x.T, dout)
#         self.db = np.sum(dout, axis=0)
        
#         return dx

#### Softmax-with-Lossレイヤ
- $n$ クラスの分類問題を考える（10種の手書き文字認識は $n = 10$）
- Softmaxレイヤへの入力は $n$ 個
- Softmaxレイヤは入力された値を正規化して出力する
- ここでは損失関数の交差エントロピー誤差を含めたレイヤを考える

***

#### Softmax関数
$$
y_k = \frac{\exp(a_k)}{\Sigma_{i=1}^n \exp(a_i)}
$$
- $n$: 分類するクラス数
- $\boldsymbol{a} = (a_1, a_2, \cdots, a_n)$: 前レイヤからの入力
- $\boldsymbol{y} = (y_1, y_2, \cdots, y_n)$: Softmax関数の出力．$\Sigma_{k=1}^{n} y_k = 1$

#### Cross Entropy Error
$$
L = - \Sigma_{k=1}^n \ t_k \log y_k
$$
- $\boldsymbol{t} = (t_1, t_2, \cdots, t_n)$: 教師ラベル．one-hot表現．

***

#### Cross Entropy Error $L$ を $y_k$ で偏微分
$$
\frac{\partial L}{\partial y_k} = - \frac{t_k}{y_k} 
$$

#### Softmax関数 $y_k$ を $a_j$ で偏微分
以下，$S = \Sigma_{i=1}^n \exp(a_i)$ とおく

次の微分を使う

$$
\begin{align}
\exp(a)^{\prime} &= \exp(a)
\\ \\
\frac{\partial S}{\partial a_j} 
&= \Sigma_{i=1}^n \frac{\partial \exp(a_i)}{\partial a_j}
\\ \\
&= \begin{cases}
\exp(a_i) & (i = j) \\
0 & (i \ne j)
\end{cases}
\end{align}
$$

商の微分公式より

$$
\frac{\partial y_k}{\partial a_j} = \frac{\exp(a_k)^{\prime} S - \exp(a_k) S^{\prime}}{S}
$$

- $j = k$ の場合
$$
\begin{align}
\frac{\partial y_k}{\partial a_k}
&= \frac{\exp(a_k) S - \exp(a_k) \exp(a_k) }{S^2} \\ \\
&= \frac{\exp(a_k)}{S} \left( 1 - \frac{\exp(a_k)}{S} \right) \\ \\
&= y_k (1 - y_k)
\end{align}
$$

- $j \ne k$ の場合
$$
\begin{align}
\frac{\partial y_k}{\partial a_j}
&= \frac{- \exp(a_k) \exp(a_j) }{S^2} \\ \\
&= - y_k y_j
\end{align}
$$

#### Cross Entropy ErrorとSoftmax関数の合成関数の微分
$$
\begin{align}
\frac{\partial L}{\partial a_k}
&= \Sigma_{i=1}^n \frac{\partial L}{\partial y_i} \frac{\partial y_i}{\partial a_k} \\ \\
&= \left(-\frac{t_1}{y_1}\right)(-y_1 y_k) + \left(-\frac{t_2}{y_2}\right)(-y_2 y_k) + \cdots + \left(-\frac{t_k}{y_k}\right) y_k(1 - y_k) + \cdots + \left(-\frac{t_n}{y_n}\right)(-y_n y_k) \\ \\
&= t_1 y_k + t_2 y_k + \cdots + t_k y_k + \cdots + t_n y_k - t_k \\ \\
&= \left(\Sigma_{i=1}^n \ t_i\right) y_k - t_k \qquad \left(\Sigma_{i=1}^n \ t_i = 1 \right) \\ \\
&= y_k - t_k
\end{align}
$$

$t_i \ (i = 1, 2, \cdots, n)$ はone-hot表現の教師データため，総和は1になる

***

#### Softmax with Lossレイヤ実装

In [18]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失
        self.y = None    # softmaxの出力
        self.t = None    # 教師データ one-hot vector
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx